In [4]:
import hoomd
import gsd.hoomd
import numpy
import freud


In [5]:
device = hoomd.device.GPU()
seed = numpy.random.randint(1,1e4)
simulation = hoomd.Simulation(device = device, seed = seed)

In [6]:
kT = 1.0
epsilon = 1.0
sigma = 1.0
final_density = 1.0
starting_density = 0.5
a = 1/(starting_density**(1/3.0))
num_replicas = 10
N_particles = num_replicas**3
grid_particles = freud.data.UnitCell([a,a,a,0,0,0],[[0,0,0]]).generate_system(num_replicas)
box_length = grid_particles[0].Lx

In [7]:
frame = gsd.hoomd.Frame()
frame.particles.N = N_particles
frame.particles.position = grid_particles[1]
frame.configuration.box = [box_length,box_length,box_length,0,0,0]

#Types of particles define different interactions. In an atomistic simulation these might be C, O, and H.
#in a coarse-grained simulation we can give them a simple name like A
frame.particles.typeid = [0]*N_particles
frame.particles.types = ['A']

#Finally, save our initial state:
with gsd.hoomd.open(name='initial_state.gsd', mode='w') as f:
    f.append(frame)

In [8]:
simulation.create_state_from_gsd(filename='initial_state.gsd')
integrator = hoomd.md.Integrator(dt = 0.005)
nve = hoomd.md.methods.NVE(filter = hoomd.filter.All())
integrator.methods.append(nve)
cell = hoomd.md.nlist.Cell(buffer=0.4)

#Define the force for different particles
lj = hoomd.md.pair.LJ(nlist=cell)

lj.params[('A', 'A')] = {"epsilon":epsilon, "sigma":sigma}

lj.r_cut[('A', 'A')] = 2.5*sigma


integrator.forces.append(lj)
simulation.operations.integrator = integrator
     
simulation.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.0)
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
    filter=hoomd.filter.All()
)

simulation.operations.computes.append(thermodynamic_properties)
logger = hoomd.logging.Logger(categories=['scalar', 'sequence'])
logger.add(simulation)
logger.add(thermodynamic_properties)
tps_tracking = hoomd.logging.Logger(categories=['scalar', 'string'])
tps_tracking.add(simulation, quantities=['timestep', 'tps'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=int(1e4)), logger=tps_tracking)
simulation.operations.writers.append(table)

/cm/shared/apps/hoomd/3.11.0/lib/python3.11/site-packages/hoomd/md/methods/methods.py:699: FutureWarning: NVE is deprecated and wil be removed in hoomd 4.0. In version 4.0, use the ConstantVolume method without a thermostat.
  warnings.warn(


In [9]:
simulation.run(1e5)

Simulation.timestep  Simulation.tps 
       10000           8749.91468   
       20000          12398.51310   
       30000          14395.10704   
       40000          15658.17046   
       50000          16534.90290   
       60000          17169.19840   
       70000          17645.43512   
       80000          18026.45686   
       90000          18336.77491   
      100000          18582.14235   
